*  整理自 https://github.com/ZhiqingXiao/rl-book/blob/master/en2024/code/CartPole-v0_OffPolicyVPGwBaseline_torch.ipynb


<mark>指定行为策略 $b$</mark>



In [1]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 13.1 MB/s eta 0:00:00


In [2]:
%matplotlib inline

import sys
import logging
import itertools  # 用于构建不限制次数的循环

import numpy as np
np.random.seed(0)
import pandas as pd
import gymnasium as gym
import matplotlib.pyplot as plt
import torch
torch.manual_seed(0)
import torch.nn as nn
import torch.optim as optim
import torch.distributions as distributions

logging.basicConfig(level=logging.INFO,
          format='%(asctime)s [%(levelname)s]%(message)s',
          stream=sys.stdout, datefmt='%H；%M:%S')

import warnings   # 忽略警告
warnings.filterwarnings("ignore")

## 环境


* https://gymnasium.farama.org/environments/classic_control/cart_pole/

In [3]:
env = gym.make('CartPole-v1', render_mode='rgb_array')  # 奖励阈值 500
print(env.action_space.n) # 动作数

2


## 算法 模块

In [4]:
class REINFORCEAgent:
  def __init__(self, env):
    self.action_n = env.action_space.n
    self.gamma = 0.99 # 折扣因子

    self.policy_net = self.build_net(
        input_size = env.observation_space.shape[0],
        hidden_sizes = [],
        output_size = self.action_n, output_activator=nn.Softmax(1))  # nn.Softmax(dim=None)： dim=0表示对列进行归一化，dim=1表示对行进行归一化
    self.policy_optimizer = optim.Adam(self.policy_net.parameters(), lr=0.06) # 学习率调大了

    ### 对 基线的估计 网络
    self.baseline_net = self.build_net(
        input_size = env.observation_space.shape[0],
        hidden_sizes = [])
    self.baseline_optimizer = optim.Adam(self.policy_net.parameters(), lr=0.1) # 学习率调大了
    self.baseline_loss = nn.MSELoss()


  def build_net(self, input_size, hidden_sizes, output_size=1,
          output_activator=None, use_bias=False):
    layers = []
    for input_size, output_size in zip([input_size,]+hidden_sizes, hidden_sizes+[output_size,]):
      layers.append(nn.Linear(input_size, output_size, bias=use_bias))
      layers.append(nn.ReLU())
    layers = layers[:-1]
    if output_activator:
      layers.append(output_activator)
    model = nn.Sequential(*layers)
    return model

  def reset(self, mode=None):
    self.mode = mode
    if self.mode == "train":
      self.trajectory = []

  def step(self, observation, reward, terminated): # 修改

    if self.mode == "train":  #
      action = np.random.choice(self.action_n) # 使用随机策略
      self.trajectory += [observation, reward, terminated, action]
    else:
      state_tensor = torch.as_tensor(observation, dtype=torch.float).unsqueeze(0)
      prob_tensor = self.policy_net(state_tensor)  #
      action = distributions.Categorical(prob_tensor).sample().item()

    return action

  def close(self):
    if self.mode == "train":
      self.learn()

  def learn(self):  # 学习 模块
    state_tensor = torch.as_tensor(self.trajectory[0::4], dtype=torch.float)
    reward_tensor = torch.as_tensor(self.trajectory[1::4], dtype=torch.float)
    action_tensor = torch.as_tensor(self.trajectory[3::4], dtype=torch.long)
    arange_tensor = torch.arange(state_tensor.shape[0],dtype=torch.float)

    #
    discount_tensor = self.gamma ** arange_tensor
    discounted_reward_tensor = discount_tensor * reward_tensor
    discounted_return_tensor = discounted_reward_tensor.flip(0).cumsum(0).flip(0)
    # 更新 baseline 网络的 参数
    return_tensor = discounted_return_tensor / discount_tensor
    pred_tensor = self.baseline_net(state_tensor)
    psi_tensor = (discounted_return_tensor - discount_tensor * pred_tensor).detach()
    baseline_loss_tensor = self.baseline_loss(pred_tensor,return_tensor.unsqueeze(1))
    self.baseline_optimizer.zero_grad()
    baseline_loss_tensor.backward()
    self.baseline_optimizer.step()

    # # 更新 策略网络的 参数
    all_pi_tensor = self.policy_net(state_tensor)
    pi_tensor = torch.gather(all_pi_tensor, 1, action_tensor.unsqueeze(1)).squeeze(1)

    # 行为网络参数
    behavior_prob = 1. / self.action_n
    policy_loss_tensor = -(psi_tensor / behavior_prob * pi_tensor).mean() #

    self.policy_optimizer.zero_grad() # 重置 梯度
    policy_loss_tensor.backward() # 反向传播
    self.policy_optimizer.step() # 更新权重

agent = REINFORCEAgent(env)

## 训练

In [ ]:
from IPython import display

def play_episode(env, agent, seed=None, mode=None, render=False):
  observation, _ = env.reset(seed=seed)
  reward, terminated, truncated = 0., False, False
  agent.reset(mode=mode)
  episode_reward, elapsed_steps = 0., 0

  if not mode and episode == 99:  # 只可视化最后一个 episode
    img = plt.imshow(env.render()) # only call this once  # 可视化

  while True:

    # Jupyter notebook  可视化 测试结果  仅显示 最后 3 个 回合
    if not mode and episode == 99:
        img.set_data(env.render()) # just update the data
        display.display(plt.gcf())
        display.clear_output(wait=True)

    action = agent.step(observation, reward, terminated)
    observation, reward, terminated, truncated, info = env.step(action)
    if render:
      env.render()
    if terminated or truncated:
      break
    episode_reward += reward
    elapsed_steps += 1 # 走过的步数
  agent.close()

  return episode_reward, elapsed_steps

logging.info("===== train =====")
episode_rewards = []
for episode in itertools.count(): # 从 0 到无穷大的 迭代次数
  episode_reward, elapsed_steps = play_episode(env, agent, seed=episode, mode="train")
  episode_rewards.append(episode_reward)
  logging.info('train episode %d：reward = %.2f, steps = %d',
               episode, episode_reward, elapsed_steps)
  if np.mean(episode_rewards[-20:]) > 200: # 获得有效的策略了  可修改得更大些，学习更好的策略，同时学习时间会变长
    break

plt.plot(episode_rewards)

logging.info("===== test =====")
episode_rewards_test = []
for episode in range(100):
  epsiode_reward, elapsed_steps = play_episode(env, agent)
  episode_rewards_test.append(episode_reward)
  logging.info('test episode % d: reward = %.2f, steps=%d',
        episode, episode_reward, elapsed_steps)
logging.info("average episode reward = %.2f ± %.2f",
       np.mean(episode_rewards_test), np.std(episode_rewards_test))

env.close()

In [ ]:
# 平滑

def moving_average(a, window_size):
    cumulative_sum = np.cumsum(np.insert(a, 0, 0))
    middle = (cumulative_sum[window_size:] - cumulative_sum[:-window_size]) / window_size
    r = np.arange(1, window_size-1, 2)
    begin = np.cumsum(a[:window_size-1])[::2] / r
    end = (np.cumsum(a[:-window_size:-1])[::2] / r)[::-1]
    return np.concatenate((begin, middle, end))

episodes_list = list(range(len(episode_rewards)))  # x
mv_return = moving_average(episode_rewards, 29)  # 9 可考虑改大些，让图好看些
plt.plot(episodes_list, mv_return)  # x y
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('REINFORCE on {}'.format(env_name))
plt.show()